<a href="https://colab.research.google.com/github/EliaFeltrin/KIND_project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name Entity Recognition

## Introduction

### Package Installation

In [ ]:
# Installing the library needed in the following part of the project

# Installing the utlity packages
#! pip install scipy
#! pip install numpy
#! pip install pandas

# Installing a natural language processing library
! pip install nltk
# Installing the packages for creating amazing plots
#! pip install matplotlib
#! pip install wordcloud
! pip install plotly
! pip install --upgrade nbformat

# Installing a package for sequence labeling, used for POS tagging and NER
! pip install -U spacy

# Installing the packages for creating the word embeddings
! pip install --upgrade gensim
! pip install fasttext

# Installing the packages for doing dimensionality reduction
#! pip install sklearn
! pip install umap-learn
! pip install python-terrier

# Installing packages for transformers
! pip install transformers==4.28.0
! pip install datasets
! pip install evaluate
! pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
 

In [ ]:
# Dowloading an italian model from spacy
! spacy download it_core_news_sm

### Package import

In [ ]:
! git clone https://github.com/EliaFeltrin/KIND_project

Cloning into 'KIND_project'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 188 (delta 83), reused 143 (delta 52), pack-reused 0
Receiving objects: 100% (188/188), 9.76 MiB | 6.51 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [ ]:
# Importing the main packages

# Importing the utlity packages
import string
import numpy as np
import random
import pandas as pd
from collections import Counter

# Importing a natural language processing library
import nltk

# Importing the packages for creating amazing plots
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px

# Importing the packages for creating the word embeddings
from gensim.models.word2vec import Word2Vec
import fasttext
import fasttext.util

# Importing the packages for doing dimensionality reduction
from sklearn.manifold import TSNE
import umap

# Importing a package for the tf-idf representation of the sentences
from sklearn.feature_extraction.text import TfidfVectorizer

# Importing a package for clustering
from sklearn.cluster import KMeans
from sklearn import metrics

# Importing the packages for POS tagging
import spacy as spc

import pyterrier as pt
if not pt.started():
  pt.init()

# Importign packages for Transformers
from datasets import Dataset, DatasetDict
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
import torch
import evaluate
import locale
locale.getpreferredencoding = lambda: "UTF-8"


terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
# Defining the names of the datasets
dataset_names = ['degasperi_train.tsv', 'degasperi_test.tsv', 'fiction_train.tsv',\
            'fiction_test.tsv', 'moro_train.tsv', 'moro_test.tsv',\
            'wikinews_train.tsv', 'wikinews_test.tsv']
          
dataset_names_train_BIO = ['degasperi_train_BIO.tsv', 'fiction_train_BIO.tsv','moro_train_BIO.tsv', 'wikinews_train_BIO.tsv']
dataset_names_test_BIO = ['degasperi_test_BIO.tsv', 'fiction_test_BIO.tsv','moro_test_BIO.tsv', 'wikinews_test_BIO.tsv']
# Defining the path to datasets
#PATH_TO_DATASETS = '../datasets/Inside_outside_NER_notation'
PATH_TO_DATASETS = '/content/KIND_project/datasets/BIO_tag_NER_notation'
# Importing all the datasets in a dictionary
#datasets_dict = {name: pd.read_csv(PATH_TO_DATASETS+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names}

datasets_train_dict_BIO = {name: pd.read_csv(PATH_TO_DATASETS+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names_train_BIO}
datasets_test_dict_BIO = {name: pd.read_csv(PATH_TO_DATASETS+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names_test_BIO}

## Function for dealing with the datasets

In this session we define some funtions useful for having the correct structure of the dataset in order to use in the various techniques we implement in the following part of the notebook.


In [ ]:
def get_string_from_df(dataframe_df, puntuaction):
  '''
  Transforms the tokenized dataset into a single string.
  It is assumed that the dataframe has two columns: one column containing the 
  tokens, so the words of the various sentences, it is named 'Tokens', and the 
  second containing the labels associated with that tokens, it is named
  'Entity'.

  Parameters
  ----------
  dataframe_df: DataFrame
    dataframe containing the tokenized dataset

  Returns
  -------
  text: str
    string concatenating all the tokens of the dataset
  '''
  text_df = dataframe_df.loc[:,'Token']
  text = text_df[0]
  for token in text_df[1:]:
    text += (' ' + token) if token not in puntuaction else token
  return text

In [ ]:
def get_sentences_list_from_df(dataset_df, key=None):
    '''
    Extracts and returns the sentences from the DataFrame given as input.
    It is assumed that the dataframe has two columns: one column containing the 
    tokens, so the words of the various sentences, it is named 'Tokens', and the 
    second containing the labels associated with that tokens, it is named
    'Entity'.

    Parameters
    ----------
    dataset_df: DataFrame
      dataframe containing the tokenized dataset
    key: list
      list containing the name of the dataset from which the dataframe comes 
      from

    Returns
    -------
    sentences_list: list

    labels_list: list

    keys_list: list


    '''

    punctuation = string.punctuation
    sentences_list = [[]]
    labels_list = [[]]
    keys_list = None
    count = 0

    for element in dataset_df.iterrows():
        if str(element[1]['Token']) == '.':
            sentences_list.append([])
            labels_list.append([])
            count += 1
        elif str(element[1]['Token']) not in punctuation:
            sentences_list[count].append(element[1]['Token'].lower())
            labels_list[count].append(element[1]['Entity'])
    if key != None:
        keys_list = [key for sentence in range(len(sentences_list))]
    return sentences_list, labels_list, keys_list

def remove_short_sentences(sentences_list, labels_list, keys_list = None, min_length=3):
    '''
    Removes the sentences with a length lower than a certain threshold

    Parameters
    ----------
    sentences_list: list
    labels_list: list
    keys_list: list
    min_lenght: int

    Returns
    -------
    sentences_list: list

    '''
    new_sentences_list = list()
    new_labels_list = list()
    new_keys_list = list()
    for idx in range(len(sentences_list)):
        if len(sentences_list[idx]) >= min_length:
            new_sentences_list.append(sentences_list[idx])
            new_labels_list.append(labels_list[idx])
            if keys_list != None:
                new_keys_list.append(keys_list[idx])
    return new_sentences_list, new_labels_list, new_keys_list

def get_all_sentences_from_datasets(datasets):
    '''
    Parameters
    ----------
    datasets: dict

    Returns
    -------
    overall_sentences: list
    overall_labels: list
    overall_keys: list

    '''
    overall_sentences = list()
    overall_labels = list()
    overall_keys = list()
    for key in datasets.keys():
        sentences, labels, keys = get_sentences_list_from_df(datasets[key], key=key)
        sentences, labels, keys = remove_short_sentences(sentences, labels, keys_list=keys)
        overall_sentences += sentences
        overall_labels += labels
        overall_keys += keys
    return overall_sentences, overall_labels, overall_keys

In [ ]:
# Defining some function useful for having the correct structure of the dataset in order to define the tf-idf representation
def concatenate_sentences_tokens(sentences):
    '''
    Parameters
    ----------
    sentences: list

    Returns
    -------
    concatenated_sentences: list

    '''
    concatenated_sentences = list()
    for sentence in sentences:
        new_sentence = ''
        for token in sentence:
            new_sentence += (token + ' ')
        concatenated_sentences.append(new_sentence)
    return concatenated_sentences

## Word embeddings representation

A word embedding is the vectorial representation of a word. It is used for achieving a dense representation of the words in an high dimensional space. Other types of representation of the words, such as the bag of words representation, are sparse compared to word embeddings.
The word embeddings allow to achieve better results in many fields of natural language processing.

In particular we use Word2Vec in order to obtain the word embeddings of out dataset.
Basic Word2Vec is a artificial neural network composed by two layers

The inputs of the neural network are the words of the sentence to convert in the word embeddings representation.
The first layer is a linear layer.
The linear activation functions values are summed and put as outputs.
At the end we appply a softmax layer.
We want the model to prefict the next word in the sentence.
I train the NN using the cross entropy as loss function.
At the end of the training the weights connecting the inputs to the first hidden layer are the values of the dimensions of the word embedding.


Two of the most used architecture of the Word2Vec are CBOW and Skip-Gram.

The Continuous Bag of Words method uses many words surrounding the word I want to use in the prediction in the training step.
The Skip-gram uses a word to predict the word in the surroundings.

In some way the distribution in the various dimension is based on the similarity of the words in terms of semantics and usage.

With word embeddings we can embed the context of the word inside its representation.


We start from the dataframe containing the tokens of the dataset.
The pipeline used for the definition of the input of the word2vec model is the following:
- (previous) the tokenization has already been done
- merging the single elements in sentences splitting on the single dots
- lowercasing all the tokens

### Word embeddings trained on all the data

We choose to try the definition of the word embeddings model on different portions of the dataset. Firstly we try to define the word embeddings on the entire data we have since the larger is the dataset the better will be the our model and the wider will be the dictionary.

In [ ]:
# Getting the sentences in the correct format to create the embeddings representation of the words
sentences, labels, sentences_keys = get_all_sentences_from_datasets(datasets_dict)
# Printing 10 lists of token
for i in range(10):
    print(str(sentences[i]))

In [ ]:
# To check that there are no tabs or new lines inside the tokens
found = False
for name in dataset_names:
    for i, el in datasets_dict[name].iterrows():
        if '\t' in el['Token'] or '\n' in el['Token']:
            found = True
            print(el['Token'])
if found:
    print('There is something wrong, there tab or new line characters, are check the import of the dataset :(')
else:
    print('No tab or new line characters found, Great job!')

In [ ]:
# Printing some information about the list of lists of token
print('The total number of sentences in the dataset is ' + str(len(sentences)))
length_list= []
for idx,i in enumerate(sentences):
    length_list.append(len(i))
print('The maximum lenght of a sentence is ' + str(max(length_list)))

In [ ]:
# Definition of the Word2Vec model
embeddings_model = Word2Vec(sentences, vector_size=30, min_count=2, window=20)
# Printing the length of the vocabulary
len(embeddings_model.wv)

In [ ]:
# Searching the most similar word to a specific word
term = 'governo'
embeddings_model.wv.most_similar(term.lower())

In [ ]:
# Computing the dimensionality reduction of the word embeddings space
word_samples = random.sample(list(embeddings_model.wv.key_to_index), 500)
word_vectors = embeddings_model.wv[word_samples]

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embeddings = tsne.fit_transform(word_vectors)
x, y, z = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
fig = px.scatter_3d(x=x, y=y, z=z, text=word_samples)
fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
fig.show()

Now it is analyzed the word embeddings trained only on the training datasets that could be useful in the next part of the project since they can be used for performing name-entity recognition. Then it is also computed and analyzed the word embeddings trained on the datasets separately.

In [ ]:
# Getting the sentences in the correct format to create the embeddings representation of the words
dataset_train_names = [key for key in dataset_names if 'train' in key]
sentences_train, labels_train = get_all_sentences_from_datasets({name:datasets_dict[name] for name in dataset_train_names})
# Definition of the Word2Vec model
embeddings_model_train = Word2Vec(sentences_train, vector_size=30, min_count=2, window=20)

In [ ]:
# Printing the length of the vocabulary
len(embeddings_model_train.wv)

In [ ]:
# Searching the most similar word to a specific word
term = 'governo'
embeddings_model_train.wv.most_similar(term.lower())

In [ ]:
# Computing the dimensionality reduction of the word embeddings space
word_samples_train = random.sample(list(embeddings_model_train.wv.key_to_index), 500)
word_vectors_train = embeddings_model_train.wv[word_samples_train]

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embeddings = tsne.fit_transform(word_vectors_train)
x_train, y_train, z_train = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
fig = px.scatter_3d(x=x_train, y=y_train, z=z_train, text=word_samples_train)
fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
fig.show()

The dictionary is a bit smaller but still big and the results are quite good. Train set and test set are usually separate and so I could create the embeddings model for the train set first and then model inside it the test set.

In [ ]:
# Getting the sentences in the correct format to create the embeddings representation of the words
dataset_train_names = [key for key in dataset_names if 'train' in key]
sentences_train_separate = {}
labels_train_separate = {}
embeddings_model_train_separate = {}
for name in dataset_train_names:
    sentences_train_sep, labels_train_sep = get_all_sentences_from_datasets({name:datasets_dict[name]})
    sentences_train_separate[name] = sentences_train_sep
    labels_train_separate[name] = labels_train_sep

    # Definition of the Word2Vec model
    embeddings_model_train_separate[name] = Word2Vec(sentences_train_sep, vector_size=30, min_count=2, window=20)

In [ ]:
# Printing the length of the vocabularies of the various datasets
for name in dataset_train_names:
    print('The dictionary of the dataset ' + name + ' is long ' + str(len(embeddings_model_train_separate[name].wv)) + ' words')

In [ ]:
# Searching the most similar word to a specific word
for name in dataset_train_names:
    term = 'governo'
    print(name + ':' + str(embeddings_model_train_separate[name].wv.most_similar(term.lower())))

It is easy to understand that the dictionaries in this case are smaller since the overall words are splitted in many models.
Trying to search the most similar embeddings to a given word in many different fields and for the different datasets we can see that better performace are achieved by the datasets that are specialized in that field, e.g. fiction_train finds worst results (less related word embeddings) for the word 'governo' than the other datasets, which deal with news and politics.

In [ ]:
# Computing the dimensionality reduction of the word embeddings space
#word_samples_train = embeddings_model_train.wv.key_to_index
#word_vectors_train = embeddings_model_train.wv[list(embeddings_model_train.wv.key_to_index)]

#tsne = TSNE(n_components=3, n_iter=1000)
#tsne_embeddings = tsne.fit_transform(word_vectors_train)
#x_train, y_train, z_train = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
#fig = px.scatter_3d(x=x_train, y=y_train, z=z_train, text=word_samples_train)
#fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
#fig.show()

In [ ]:
# TO DOOOOOOO

#fig = plt.figure(figsize=(15, 10))
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(x, y, z, c=LabelEncoder().fit_transform(labels), marker='.');

## Computing the embeddings using a pretrained model FastText

In [ ]:
#!wget http://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
#!gzip -d cc.en.300.bin.gz

In [ ]:
#ft = fasttext.load_model('cc.en.300.bin')

## Clustering

The clustering is an unsupervided machine learning technique useful for inspecting not labeled data in order to find inside them hidden patterns or other information.
In our case clustering is not needed for performing name-entity recognition but it is useful for gain some additional knowledge about the dataset before proceeding in the sequence labeling task.

Maybe we could improve in some way the NER applying it to some specific cluster? bohhh


Since it is not possible to identify the various documents we decide to apply the clustering techniqes on the sentences. We assume that the sentences were separate by the dot and we extract in this way them from the input files.
We can investigate whether the entities creates some cluster or not.
We can also investigate whether the four dataset are distinguishable and so in some way they have different characteristics.

We decide to try apply the clustering on the datasets and see whether the classification gives us good performances.

In [ ]:
# Importing the italian stopwords (taken from https://github.com/stopwords-iso/stopwords-it.git)
with open('stopwords-it.txt', 'r') as f:
    italian_stopwords = f.read()
italian_stopwords_github = italian_stopwords.split('\n')
len(italian_stopwords_github)

In [ ]:
# Downloading and importing the italian stopwords in the package nltk
nltk.download('stopwords')
italian_stopwords_nltk = nltk.corpus.stopwords.words('italian')
len(italian_stopwords_nltk)

In [ ]:
# All the stopwords od nltk are in the other dataset, better result with more stopwords
count = 0
for i in italian_stopwords_nltk:
    if i in italian_stopwords_github:
        count += 1
print(count)
italian_stopwords = italian_stopwords_github

In [ ]:
# Defining the vectorizer model
vectorizer = TfidfVectorizer(max_df=0.5, min_df=3, stop_words=italian_stopwords_github, use_idf=True)
# Fitting the vectorizer model
vectorizer.fit(concatenate_sentences_tokens(sentences))

# Given that we are evaluating sentences the frequncy can be lower for saying that we have a stopword

In [ ]:
# Extracting the vocabulary
vocabulary = vectorizer.get_feature_names_out()

print('The vocabulary is long ' + str(len(vocabulary)) + ' words')

In [ ]:
# The words look reasonable and the vocabulary seems to not have many stopwords inside
sorted(random.sample(vocabulary.tolist(),100))

In [ ]:
# Converting the strings into vectors
sentences_vector = vectorizer.transform(concatenate_sentences_tokens(sentences))

In [ ]:
sentences_vector[0].multiply(sentences_vector[0]).sum()

In [ ]:
max_score = 0
sentence_idx = 1000
for i in range(sentences_vector.shape[0]):
    if i != sentence_idx:
        score = sentences_vector[sentence_idx].multiply(sentences_vector[i]).sum()
        if score > max_score:
            max_score = score
            max_score_idx = i
print('Sentence index ' + str(sentence_idx) + ': ' + str(sentences[sentence_idx]))
print('Most similar sentence is the one with index ' + str(max_score_idx) + ': ' + str(sentences[max_score_idx]))
print('The score is ' + str(max_score))

The sentences obtain not so good metrics of similarity, the retrieved sentences have some similarities, same words, but they are not very similar in the semantics or in the field of application of the sentence.

Here we try to cluster the sentences in the 4 datsets, so we expect four clasters, we use the K-means clustering method.

In [ ]:
# Defining the labels of the clustering
keys_datasets = list()
for key in sentences_keys:
    keys_datasets.append(key.replace('_train.tsv', '').replace('_test.tsv', ''))
# Applying the K-means clustering
num_clusters= len(set(keys_datasets))
kmeans_model = KMeans(n_clusters=num_clusters, max_iter=1000, n_init=2, verbose=True, random_state=2307)
kmeans_model.fit(sentences_vector)

In [ ]:
for i in range(num_clusters):
    centroid = kmeans_model.cluster_centers_[i]
    sorted_terms = centroid.argsort()[::-1]
    print('Centroid of cluster ' + str(i))
    print([vocabulary[j] for j in sorted_terms[:20]])

In [ ]:
print("Top terms per cluster:")
vocabulary = vectorizer.get_feature_names_out()

for i in range(kmeans_model.n_clusters):
    centroid = kmeans_model.cluster_centers_[i]
    sorted_terms = centroid.argsort()[::-1]
    print(f"Cluster {i}:\t{[vocabulary[j] for j in sorted_terms[:10]]}")

In [ ]:
print('Number of sentences in: ')

for i in range(kmeans_model.n_clusters):
    print(f"Cluster {i}: {np.sum(kmeans_model.labels_ == i)}")

In [ ]:
print("Intrinsic evaluation measures:")
print("Within-cluster sum-of-squares:", str(kmeans_model.inertia_))
print("Silhouette coefficient:", str(metrics.silhouette_score(sentences_vector, kmeans_model.labels_)))

Very bad results :(

In [ ]:
print('Extrinsic evaluation measures:')
print("Homogeneity:", str(metrics.homogeneity_score(keys_datasets, kmeans_model.labels_)))
print("Completeness:", str(metrics.completeness_score(keys_datasets, kmeans_model.labels_)))
print("V-measure:", str(metrics.v_measure_score(keys_datasets, kmeans_model.labels_)))
print("Adjusted Rand-Index:", str(metrics.adjusted_rand_score(keys_datasets, kmeans_model.labels_)))

## POS tagging

In [ ]:
# Loading the italian model
nlp_model = it_core_news_sm.load()

text_degasperi = ' '.join(dataset_degasperi.iloc[:, 0].tolist())
parsed_text_degasperi = nlp_model(text_degasperi)

text_fiction = ' '.join(dataset_fiction.iloc[:, 0].tolist())
parsed_text_fiction = nlp_model(text_fiction)

text_wikinews = ' '.join(dataset_wikinews.iloc[:, 0].tolist())
parsed_text_wiki = nlp_model(text_wikinews[:1000000])

text_moro = ' '.join(dataset_moro.iloc[:, 0].tolist())
parsed_text_moro = nlp_model(text_moro[:1000000])

In [ ]:
def merge_counters(counter1, counter2):

  '''
  Takes 2 counters with different shapes and in the smallest one ad also the key that are currently inside with a value of 0

  Parameters
  ----------
  counter1: Counter
    Counter with smallest shape that has to be incremented
  counter2: Counter
    Counter with the larger shape

  Returns
  -------
  new_counter: Counter
    Counter containing all the tuple of the smallest one and the tuples (key, 0) of the larger one that were no present in the small one
  '''
  new_counter = counter1    

  for key, value in counter2.items():
    if key not in new_counter.keys():
      new_counter[key] = 0 

  return new_counter

In [ ]:
def plot_wordCloud_counters(counters):

  '''
  Takes as input a list of counters and it plots the wordCloud

  Parameters
  ----------
  cunters: list(Counter)
    List of counters that has to be plotted. It does not require that all the counters has the same shape

  '''

  word_cloud_counter = Counter()
  for counter in list_counters:
    word_cloud_counter.update(counter)
  # Generate a word cloud from the POS counts
  wordcloud = WordCloud(background_color='white').generate_from_frequencies(word_cloud_counter)

  # Plot the word cloud
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.show()

In [ ]:
def plot_groupedBar_counters(counters):

  '''
  Takes as input a list of counters and it plots in the same bar chart the counts

  Parameters
  ----------
  cunters: list(Counter)
    List of counters that has to be plotted. It does not require that all the counters has the same shape

  '''
  
  max_length = max(map(len, list_counters))                         # max length of the counters
  max_position = list(map(len, list_counters)).index(max_length)    # position in the list of the counter with max length

  # For each counter that is not the one of maximum dimension I merge it with all the other ones. The result is a list with counters having all the same keys
  for i in range(len(list_counters)):
    if i != max_position:
      list_counters[i] = merge_counters(list_counters[i], list_counters[max_position])

  # We plot each counter inside the bar chart
  x = np.arange(max_length)
  width=0.2
  multiplier = 0
  for counter in list_counters:
    offset = width * multiplier
    labels, values = zip(*sorted(counter.items()))
    plt.bar(x + offset, values, width=width)
    multiplier += 1

  plt.title("POS Tag Frequency Distribution Degasperi")
  plt.xlabel("POS Tag")
  plt.ylabel("Frequency")
  plt.xticks(x + width, sorted(list_counters[max_position]), rotation='vertical')
  plt.show()

In [ ]:
# Get the frequency distribution of POS tags
pos_freq_degasperi = Counter([token.pos_ for token in parsed_text_degasperi])
pos_freq_fiction = Counter([token.pos_ for token in parsed_text_fiction])
pos_freq_wiki = Counter([token.pos_ for token in parsed_text_wiki])
pos_freq_moro = Counter([token.pos_ for token in parsed_text_moro])

list_counters = [pos_freq_degasperi, pos_freq_fiction, pos_freq_wiki, pos_freq_moro]
plot_groupedBar_counters(list_counters)
plot_wordCloud_counters(list_counters)

## Indexing

Indexing is the process by which search engines organize information before a search to enable super-fast responses to queries. 
Searching through individual pages for keywords and topics would be a very slow process for search engines to identify relevant information. Instead, search engines use an inverted index, also known as a reverse index.
An inverted index is a system wherein a database of text elements is compiled along with pointers to the documents which contain those elements



First of all, since we don't have a dataset of documents we decided to index the sentences inside our datasets.
To do that we have to create the sentences from the dictionary of datasets.

In [ ]:
sentences, labels, keys = get_all_sentences_from_datasets(datasets_dict)

Then, we create the dataset of sentences adding also the Id to indetify them while the query part.

In [ ]:
sentences_df = pd.DataFrame(columns=['docno', 'text'])

for i in range(len(sentences)):
    sentences_df.loc[i] = [f's{i}', ' '.join(sentences[i])]

In the following cell, we index the sentences dataframe. The index, with all its data structures, is written into a directory that we will call `index`.
`Index_Ref` provides the location where the index is stored.

In [ ]:
index_path = "./index"

indexer = pt.DFIndexer(index_path, overwrite=True)
index_ref = indexer.index(sentences_df['text'], sentences_df['docno'])
index_ref.toString()

Now we can now load the index and print it.
This is a Terrier index structure, which provides methods such as:
 - `getCollectionStatistics()`
 - `getInvertedIndex()`
 - `getLexicon()`

 Let's see what is returned by the `CollectionStatistics()` method.

In [ ]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Now that we have indexed our documents, we can run a search over the document collection.
Here we used the TF-IDF weighting formula to rank the results. 

The `search()` method returns a dataframe with columns:
 - `qid`: the query identifier
 - `docid`: integer identifier for document 
 - `docno`: string identifier for document
 - `rank`: rank position
 - `score`: tf-idf score
 - `query`: the input query

In [ ]:
br = pt.BatchRetrieve(index, wmodel="TF_IDF")

queries = pd.DataFrame([["query1", "festa"], ["query2", "ammiraglio"], ["query3", "messaggio audio"]], columns=["qid", "query"])
br(queries)

## Conditional random fields

Conditinal random fields can be seen as an extention of the Hidden Markov Models.

HMM take a sequence of data, in our case a sequence of tokens (words), and assumes that the observations are given by a set of state that we don't observe but which we are interested to tag. There are transition probabilities between the states in the sequence (probability of going into a specific next state) and there are emission probability for each state that states which is the probability of a observation in a given state.

HMM is a generative model: it aim to model the joint probability of hidden state and observation simultaniously. Framework in which we understand the way the data we have are generated.

HMM has some limitation:
- transition and emission probabilities are static, in every position of the sentence in our case (we expect that the probabilities can change in different positions of a sentence?);
- each state depends only on the previous state but in natural language processing this is not the case, also the observations can depend on previous states in the speech (but we can imagine the dependency can be in the current state).

CRF
CRFs are a type of discriminative undirected probabilistic graphical model.
We can imagine this model as an extension of the HMM where we can draw whatever dependency we want between the states. We have an undirected graph where the nodes are the states. The states are connected with each other in an arbitrary way and a states depends on the state to which it is connected through an edge.

More general framework for catching the dependencies between the state and also between states and observations.

We want to find the probability of the state conditioned the observations. We don't find the process generation. In our case we want to discriminate whether a token is part of an entity or not given the sentence.
Often we use a variant of the method called linear chain CRF, in case we have a big dataset. In case of LCCRF we consider associations between states that are next to each other (consequent) in time and we consider relations with observations and all the hidden states.

The underlying idea is the one of the logistic regression.

--- continue ---

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
from sklearn_crfsuite import CRF

In [ ]:
#Creating the CRF model for NER
ner_crf_model = CRF(c1=0.1, c2=0.1, max_iterations=100,  all_possible_transitions=False)

In [ ]:
# Fitting the CRF model using the training set
ner_crf_model.fit()

## NER Transformers


In [ ]:
def createDataset(train, test):
  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)
  dataset_hf = DatasetDict()
  
  dataset_hf['train'] = train_dataset
  dataset_hf['test'] = test_dataset

  return dataset_hf

In [ ]:
datasets_merged = pd.concat(datasets_train_dict_BIO.values(), ignore_index=True)
entity_names = pd.unique(datasets_merged['Entity'])
entity_names_dict = {}
for i, label in enumerate(entity_names):
  entity_names_dict[label] = i

lables2ids = {}
for i, label in enumerate(entity_names):
  lables2ids[i] = label

sentences, labels, keys = get_sentences_list_from_df(datasets_merged)
train_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])

for i in range(len(sentences)):
  train_dataset.at[i, 'Tokens'] = sentences[i]
  train_dataset.at[i, 'Labels'] = labels[i]

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'dbmdz/bert-base-italian-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(entity_names), label2id=lables2ids, id2label=entity_names_dict).to(device)

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["Tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['Labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(entity_names_dict[label[word_idx]])
            else:
                label_ids.append(entity_names_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    entity_metrics = {}
    for entity_name in entity_names:
        true_predictions = [[entity_names[p] for (p, l) in zip(prediction, label) if l != -100 and entity_names[l] == entity_name] for prediction, label in zip(predictions, labels)]
        true_labels = [[entity_names[l] for (p, l) in zip(prediction, label) if l != -100 and entity_names[l] == entity_name] for prediction, label in zip(predictions, labels)]

        results = metric.compute(predictions=true_predictions, references=true_labels)
        entity_metrics[entity_name] = {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

    return entity_metrics

In [ ]:
def compute_mean_metrics_cv(results):
  metrics = {}
  for entity_name in entity_names:
    metrics[entity_name] = {
            "precision": sum([scores[f'eval_{entity_name}']['precision'] for scores in results]) / len(results),
            "recall":  sum([scores[f'eval_{entity_name}']['recall'] for scores in results]) / len(results),
            "f1":  sum([scores[f'eval_{entity_name}']['f1'] for scores in results]) / len(results),
            "accuracy":  sum([scores[f'eval_{entity_name}']['accuracy'] for scores in results]) / len(results)
        }
  return metrics

In [ ]:
learning_rates = [2e-5, 3e-5]
epochs = [2,3,4]

data_collator = DataCollatorForTokenClassification(tokenizer)

results = []
for lr in learning_rates:
  for epoch in epochs:
    results_cv = []
    for i in range(3):
      train, val = train_test_split(train_dataset, test_size=0.2, shuffle=True)
      dataset_hf = createDataset(train, val)
      tokenized_datasets = dataset_hf.map(tokenize_and_align_labels, batched=True)

      args = TrainingArguments(output_dir='ner',
                               evaluation_strategy = "no",
                               learning_rate=lr,
                               per_device_train_batch_size=32,
                               per_device_eval_batch_size=32,
                               num_train_epochs=epoch,
                               weight_decay=0.01,
                               push_to_hub=False,
                               disable_tqdm=False
                              )

      trainer = Trainer(
          model,
          args,
          train_dataset=tokenized_datasets['train'],
          eval_dataset = tokenized_datasets['test'],
          data_collator=data_collator,
          compute_metrics=compute_metrics,
          tokenizer=tokenizer,
      )
      trainer.train()
      results_cv.append(trainer.evaluate())
    results.append(compute_mean_metrics_cv(results_cv))

Map:   0%|          | 0/22030 [00:00<?, ? examples/s]

Map:   0%|          | 0/5508 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.133800
1000,0.056400


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.993237614544029}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8979676482787226, 'recall': 0.7881325081907535, 'f1': 0.8394726638231873, 'accuracy': 0.7975973789588642}"

...

Map:   0%|          | 0/22030 [00:00<?, ? examples/s]

Map:   0%|          | 0/5508 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.001100
1000,0.001000
1500,0.001200
2000,0.001200
2500,0.000400


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9963275931843029}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9462698105914187, 'recall': 0.8577435178696566, 'f1': 0.899834589229921, 'accuracy': 0.8622985283812193}"

In [ ]:
results

[{'O': {'precision': 0.0,
   'recall': 0.0,
   'f1': 0.0,
   'accuracy': 0.9953177341135081},
  'B-ORG': {'precision': 0.9277937415859832,
   'recall': 0.8469201329568338,
   'f1': 0.8853566819345958,
   'accuracy': 0.8531117635539124},
  'I-ORG': {'precision': 0.7423604331569077,
   'recall': 0.8132441562740346,
   'f1': 0.7760856379683423,
   'accuracy': 0.8259510660284105},
  'B-LOC': {'precision': 0.9402673581714179,
   'recall': 0.9003465779355079,
   'f1': 0.9198291423698084,
   'accuracy': 0.9054995629122408},
  'B-PER': {'precision': 0.9771718708046574,
   'recall': 0.9355670970677384,
   'f1': 0.9558953773324647,
   'accuracy': 0.9422635936103231},
  'I-LOC': {'precision': 0.6359661891879052,
   'recall': 0.7745397069503183,
   'f1': 0.6974346779571752,
   'accuracy': 0.8320160326886784},
  'I-PER': {'precision': 0.8439765832210865,
   'recall': 0.9192754193686508,
   'f1': 0.8799522968588275,
   'accuracy': 0.9506929661466054}},
 {'O': {'precision': 0.0,
   'recall': 0.0,
   

In [ ]:
dataset_hf = Dataset.from_pandas(train_dataset)
tokenized_dataset = dataset_hf.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(output_dir='ner',
                          evaluation_strategy = "no",
                          learning_rate=2e-5,
                          per_device_train_batch_size=32,
                          per_device_eval_batch_size=32,
                          num_train_epochs=4,
                          weight_decay=0.01,
                          push_to_hub=False,
                          disable_tqdm=False
                        )

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Map:   0%|          | 0/27538 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.012800
1000,0.009800
1500,0.005400
2000,0.003600
2500,0.002800
3000,0.001900


TrainOutput(global_step=3444, training_loss=0.0054789323632310075, metrics={'train_runtime': 527.7833, 'train_samples_per_second': 208.707, 'train_steps_per_second': 6.525, 'total_flos': 5870721619483776.0, 'train_loss': 0.0054789323632310075, 'epoch': 4.0})

In [ ]:
results_test = []
for test in datasets_test_dict_BIO.values():
  sentences, labels, keys = get_sentences_list_from_df(test)
  test_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])
  for i in range(len(sentences)):
    test_dataset.at[i, 'Tokens'] = sentences[i]
    test_dataset.at[i, 'Labels'] = labels[i]
    
  test_dataset = Dataset.from_pandas(test_dataset)
  test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
  results_test.append(trainer.evaluate(test_dataset))

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9937352594339622}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8801089918256131, 'recall': 0.6632443531827515, 'f1': 0.7564402810304449, 'accuracy': 0.6673511293634496}

Map:   0%|          | 0/807 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9932994228699991}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8191881918819188, 'recall': 0.7449664429530202, 'f1': 0.7803163444639718, 'accuracy': 0.738255033557047}"

Map:   0%|          | 0/2203 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9971716567612355}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9781512605042016, 'recall': 0.8885496183206106, 'f1': 0.9312, 'accuracy': 0.8916030534351145}" of type <c

Map:   0%|          | 0/1804 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9902612095650127}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8974654377880185, 'recall': 0.7701433514582303, 'f1': 0.8289438680500133, 'accuracy': 0.7745921898171033}

In [ ]:
datasets_test_merged = pd.concat(datasets_test_dict_BIO.values(), ignore_index=True)
sentences, labels, keys = get_sentences_list_from_df(datasets_test_merged)
test_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])
for i in range(len(sentences)):
  test_dataset.at[i, 'Tokens'] = sentences[i]
  test_dataset.at[i, 'Labels'] = labels[i]
test_dataset = Dataset.from_pandas(test_dataset)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
trainer.evaluate(test_dataset)

Map:   0%|          | 0/5648 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.994144764151974}" of type <class 'dict'> for key "eval/O" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9043448972718087, 'recall': 0.7753393011839446, 'f1': 0.8348880597014926, 'accuracy': 0.7785157377995957}"

{'eval_loss': 0.10658223181962967,
 'eval_O': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'accuracy': 0.994144764151974},
 'eval_B-ORG': {'precision': 0.9043448972718087,
  'recall': 0.7753393011839446,
  'f1': 0.8348880597014926,
  'accuracy': 0.7785157377995957},
 'eval_I-ORG': {'precision': 0.7853185595567868,
  'recall': 0.8031161473087819,
  'f1': 0.7941176470588236,
  'accuracy': 0.7972064148991206},
 'eval_B-LOC': {'precision': 0.9346664026925361,
  'recall': 0.8847451274362819,
  'f1': 0.9090208914989891,
  'accuracy': 0.8901799100449775},
 'eval_B-PER': {'precision': 0.9701851406588122,
  'recall': 0.9020791415157612,
  'f1': 0.9348934198331789,
  'accuracy': 0.9047619047619048},
 'eval_I-LOC': {'precision': 0.6605504587155964,
  'recall': 0.768,
  'f1': 0.7102342786683108,
  'accuracy': 0.7864583333333334},
 'eval_I-PER': {'precision': 0.8333333333333334,
  'recall': 0.9206081081081081,
  'f1': 0.8747993579454254,
  'accuracy': 0.9412972085385879},
 'eval_runtime': 17.

# CODICE ELIA

# KIND DATASET

## Brief Description
KIND (Kessler Italian Named-entities Dataset) is a dataset released in 2022 by researchers from Fondazione Bruno Kessler and the University of Trento. It contains 1 million tokens, of which 600K name-entities are manually annotated. The entities belong to 3 classes (people, location, organization). The texts come from various sources of the Italian language, such as news articles, literature, and political speeches, making it a multi-domain dataset. 
The following table shows the dataset composition:

<table>
  <thead>
    <tr>
      <th rowspan="2">Dataset</th>
      <th rowspan="2">Documents</th>
      <th rowspan="2">Tokens</th>
      <th colspan="4">Train</th>
      <th colspan="4">Test</th>
    </tr>
    <tr>
      <th>Total</th>
      <th>PER</th>
      <th>ORG</th>
      <th>LOC</th>
      <th>Total</th>
      <th>PER</th>
      <th>ORG</th>
      <th>LOC</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Wikinews</td>
      <td>1,000</td>
      <td>308,622</td>
      <td>247,528</td>
      <td>8,928</td>
      <td>7,593</td>
      <td>6,862</td>
      <td>61,094</td>
      <td>1,802</td>
      <td>1,823</td>
      <td>1,711</td>
    </tr>
    <tr>
      <td>Fiction</td>
      <td>86</td>
      <td>192,448</td>
      <td>170,942</td>
      <td>3,439</td>
      <td>182</td>
      <td>733</td>
      <td>21,506</td>
      <td>636</td>
      <td>284</td>
      <td>463</td>
    </tr>
    <tr>
      <td>Aldo Moro</td>
      <td>250</td>
      <td>392,604</td>
      <td>309,798</td>
      <td>1,459</td>
      <td>4,842</td>
      <td>2,024</td>
      <td>82,806</td>
      <td>282</td>
      <td>934</td>
      <td>807</td>
    </tr>
    <tr>
      <td>Alcide De Gasperi</td>
      <td>158</td>
      <td>150,632</td>
      <td>117,997</td>
      <td>1,129</td>
      <td>2,396</td>
      <td>1,046</td>
      <td>32,635</td>
      <td>253</td>
      <td>533</td>
      <td>274</td>
    </tr>
    <tr>
      <td><strong>Total</strong></td>
      <td><strong>1494</strong></td>
      <td><strong>1,044,306</strong></td>
      <td><strong>846,265</strong></td>
      <td><strong>14,955</strong></td>
      <td><strong>15,013</strong></td>
      <td><strong>10,665</strong></td>
      <td><strong>198,041</strong></td>
      <td><strong>2,973</strong></td>
      <td><strong>3,574</strong></td>
      <td><strong>3,255</strong></td>
    </tr>
  </tbody>
</table>


##Annotation Tagging scheme
The tokens of all datasets, except for Aldo Moro, have been manually labeled using the IOB (Inside-Outside-Beginning) convention: each entity is labeled as begin-of-entity (B-[ent]) or continuation-of-entity (I-[ent]). The annotations of the Aldo Moro dataset, instead, were carried out with a mixed process that used both manual and automatic annotations (subsequently checked by hand); due to some differences in the convention for annotation, this dataset does not contain information for composite entities (beginning, continuation). For more details, please refer to the paper related with the dataset release: https://arxiv.org/abs/2112.15099

An example of the annotations is reported here: 


# IOB annotation (from ADG_dev.tsv)
Nel	O
nostro	O
Trentino	B-LOC
attraversiamo	O
un	O
momento	O
storico	O
importante	O
.	O

# non-IOB annotation (from moro_test.tsv)
Dal	O
Consiglio	ORG
nazionale	ORG
del	O
‘	O
75	O
la	O
grande	O
stampa	O
parla	O
di	O
due	O
anime	O
contrapposte	O
del	O
partito	O
.	O

CODICE PER CHARTS. RIMUOVO LE MBRERIE IN SEGUITO

In [ ]:
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from matplotlib import rc



BIO_PATH = '../datasets/BIO_tag_NER_notation/'
nonBIO_PATH = '../datasets/Inside_outside_NER_notation/'


SPIDER_ROW_N = 2
SPIDER_COL_N = 2
BAR_ROW_N = 1 
BAR_COL_N = 2

def to_lowerCase(df):
    return pd.DataFrame({'Token': df['Token'].str.lower(), 'Entity': df['Entity']})

def add_column_names(df):
    return  df.rename(columns={0: 'Token', 1: 'Entity'})

def spider_plot(df, group, title, subplot_idx):
    plt.rcParams["figure.figsize"] = (10,10)
    categories=list(df)[:]
    N = len(categories)
    
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    
    # Initialise the spider plot
    ax = plt.subplot(SPIDER_ROW_N, SPIDER_COL_N, subplot_idx, polar=True)
    
    # first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    
    # Draw one axe per variable + add labels
    plt.xticks(angles[:-1], categories)
    
    # Draw ylabels
    #ax.set_yscale('log')
    ax.set_rlabel_position(0)
    min = df.min().min()
    max = df.max().max()
    plt.ylim(min -(max-min)/10, max + (max-min)/10)
 
    # Plot each individual = each line of the data
    # I don't make a loop, because plotting more than 3 groups makes the chart unreadable
    
    for i in range(len(group)):
        values=df.loc[i].values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=group[i])
        ax.fill(angles, values, 'b', alpha=0.1)

    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title(title)

    # Show the graph
    #plt.show()

############################################################### reading datasets ###############################################################

ds = {'ds_mr'           : pd.read_csv(nonBIO_PATH + 'moro_train.tsv', sep='\t', header=None),
      'ds_mr_test'      : pd.read_csv(nonBIO_PATH + 'moro_test.tsv', sep='\t', header=None),
      
      'ds_dg_IOB'       : pd.read_csv(BIO_PATH + 'degasperi_train_BIO.tsv', sep='\t', header=None),
      'ds_fc_IOB'       : pd.read_csv(BIO_PATH + 'fiction_train_BIO.tsv', sep='\t', header=None),
      'ds_wn_IOB'       : pd.read_csv(BIO_PATH + 'wikinews_train_BIO.tsv', sep='\t', header=None),
      'ds_dg_IOB_test'  : pd.read_csv(BIO_PATH + 'degasperi_test_BIO.tsv', sep='\t', header=None),
      'ds_fc_IOB_test'  : pd.read_csv(BIO_PATH + 'fiction_test_BIO.tsv', sep='\t', header=None),
      'ds_wn_IOB_test'  : pd.read_csv(BIO_PATH + 'wikinews_test_BIO.tsv', sep='\t', header=None)
}

############################################################### calculating stats ###############################################################
stats = {}

for i in ds.keys():     
    ds[i] = add_column_names(ds[i])
    ds[i] = to_lowerCase(ds[i])

    stats[i] = {
        'doc_len' : ds[i]['Token'].count(),
        'voc_size' : ds[i]['Token'].nunique(),
        'n_punct': sum(1 for k in ds[i]['Token'] if all(char in string.punctuation for char in k))
    }

    if('IOB' in str(i)):
        
        stats[i]['n_I-PER'] = sum(1 for k in ds[i]['Entity'] if k == 'I-PER') 
        stats[i]['n_I-ORG'] = sum(1 for k in ds[i]['Entity'] if k == 'I-ORG')
        stats[i]['n_I-LOC'] = sum(1 for k in ds[i]['Entity'] if k == 'I-LOC')

        stats[i]['n_B-PER'] = sum(1 for k in ds[i]['Entity'] if k == 'B-PER') 
        stats[i]['n_B-ORG'] = sum(1 for k in ds[i]['Entity'] if k == 'B-ORG')
        stats[i]['n_B-LOC'] = sum(1 for k in ds[i]['Entity'] if k == 'B-LOC')

        stats[i]['n_PER'] = stats[i]['n_I-PER'] + stats[i]['n_B-PER']
        stats[i]['n_ORG'] = stats[i]['n_I-ORG'] + stats[i]['n_B-ORG']
        stats[i]['n_LOC'] = stats[i]['n_I-LOC'] + stats[i]['n_B-LOC']
        
    else:
        stats[i]['n_PER'] = sum(1 for k in ds[i]['Entity'] if k == 'PER') 
        stats[i]['n_ORG'] = sum(1 for k in ds[i]['Entity'] if k == 'ORG')
        stats[i]['n_LOC'] = sum(1 for k in ds[i]['Entity'] if k == 'LOC')

    stats[i]['n_O'] = sum(1 for k in ds[i]['Entity'] if k == 'O')
        

avg_doc_len = sum(stats[i]['doc_len'] for i in stats.keys()) / len(stats.keys())
avg_voc_size = sum(stats[i]['voc_size'] for i in stats.keys()) / len(stats.keys())

############################################################### preparing data for plotting ###############################################################
 
# Values of each group
iper = [stats[i]['n_I-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iorg = [stats[i]['n_I-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iloc = [stats[i]['n_I-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]

bper = [stats[i]['n_B-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
borg = [stats[i]['n_B-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
bloc = [stats[i]['n_B-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]

iob_punct = [stats[i]['n_punct'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iob_o = [stats[i]['n_O'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iob_o = [iob_o[i] - iob_punct[i] for i in range(len(iob_o))]

iper_test = [stats[i]['n_I-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iorg_test = [stats[i]['n_I-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iloc_test = [stats[i]['n_I-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]

bper_test = [stats[i]['n_B-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
borg_test = [stats[i]['n_B-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
bloc_test = [stats[i]['n_B-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]

iob_punct_test = [stats[i]['n_punct'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iob_o_test = [stats[i]['n_O'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iob_o_test = [iob_o_test[i] - iob_punct_test[i] for i in range(len(iob_o_test))]

iob_voc_size = [stats[i]['voc_size'] for i in stats.keys() if 'IOB' in str(i)]

per = [stats[i]['n_PER'] for i in stats.keys() if 'test' not in str(i)]
org = [stats[i]['n_ORG'] for i in stats.keys() if 'test' not in str(i)]
loc = [stats[i]['n_LOC'] for i in stats.keys() if 'test' not in str(i)]

per_test = [stats[i]['n_PER'] for i in stats.keys() if 'test' in str(i)]
org_test = [stats[i]['n_ORG'] for i in stats.keys() if 'test' in str(i)]
loc_test = [stats[i]['n_LOC'] for i in stats.keys() if 'test' in str(i)]

voc_size = [stats[i]['voc_size'] for i in stats.keys() if 'test' not in str(i)]
voc_size_test = [stats[i]['voc_size'] for i in stats.keys() if 'test' in str(i)]

doc_len = [stats[i]['doc_len'] for i in stats.keys() if 'test' not in str(i)]
doc_len_test = [stats[i]['doc_len'] for i in stats.keys() if 'test' in str(i)]

n_punct = [stats[i]['n_punct'] for i in stats.keys() if 'test' not in str(i)]
n_punct_test = [stats[i]['n_punct'] for i in stats.keys() if 'test' in str(i)]

o = [stats[i]['n_O'] for i in stats.keys() if 'test' not in str(i)]
punct = [stats[i]['n_punct'] for i in stats.keys() if 'test' not in str(i)]


o_test = [stats[i]['n_O'] for i in stats.keys() if 'test' in str(i)]
punct_test = [stats[i]['n_punct'] for i in stats.keys() if 'test' in str(i)]

############################################################### spider plots ###############################################################

spider_plot( pd.DataFrame({
    'B-PER': bper,
    'I-PER': iper,
    'B-ORG': borg,
    'I-ORG': iorg,
    'B-LOC': bloc,
    'I-LOC': iloc }),
    ['deGasperi', 'Fiction', 'Wikinews'],
    'Train set IOB tags',
    1)

spider_plot( pd.DataFrame({
    'B-PER': bper_test,
    'I-PER': iper_test,
    'B-ORG': borg_test,
    'I-ORG': iorg_test,
    'B-LOC': bloc_test,
    'I-LOC': iloc_test }),
    ['deGasperi', 'Fiction', 'Wikinews'],
    'Test set IOB tags',
    2)

spider_plot( pd.DataFrame({
    'B-PER': per,
    'B-ORG': org,
    'B-LOC': loc}),
    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
    'Train set non-IOB tags',
    3)

spider_plot( pd.DataFrame({
    'B-PER': per_test,
    'B-ORG': org_test,
    'B-LOC': loc_test}),
    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
    'Test set non-IOB tags',
    4)

#spider_plot( pd.DataFrame({
#    'doc_len': doc_len,
#    'voc_size': voc_size,
#    'n_punct': n_punct,}),
#    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
#    'Train set document statistics',
#    5)
#
#spider_plot( pd.DataFrame({
#    'doc_len': doc_len_test,
#    'voc_size': voc_size_test,
#    'n_punct': n_punct_test,}),
#    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
#    'Test set document statistics',
#    6)

plt.show()


def stacked_bar_plot(col_names, data, labels_name, title, subplot_idx):
    plt.rcParams["figure.figsize"] = (15,10)
    # Names of group and bar width
    barWidth = 1
    bars = np.zeros(len(col_names))
    n_col = np.arange(len(col_names))
    plt.subplot(BAR_ROW_N, BAR_COL_N, subplot_idx)


    for i in range(len(data)):
        plt.bar(n_col, data[i], bottom=bars, edgecolor='white', width=barWidth, label=labels_name[i])
        bars = np.add(bars, data[i]).tolist()

    bars = np.add(bper, iper).tolist()
    
    # Custom X axis
    plt.xticks(n_col, col_names, fontweight='bold')
    plt.legend()
    plt.title(title)

    
    # Show graphic
stacked_bar_plot(['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
                 [punct, [o[i]-punct[i] for i in range(len(o))]],
                 ['punct', 'O'],
                 'Train sets',
                 1)

stacked_bar_plot(['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
                 [punct_test, [o_test[i]-punct_test[i] for i in range(len(o_test))]],
                 ['punct', 'O'],
                 'Test sets',
                 2)

plt.show()

In [ ]:
import pandas as pd
import os

#BIO_PATH = 'BIO_tag_NER_notation'
#nonBIO_PATH = 'Inside_outside_NER_notation'

#function to check if 2 tsv files are equal
def check_equal(file1, file2):
	df1 = pd.read_csv(file1, sep='\t')
	df2 = pd.read_csv(file2, sep='\t')
	#printf the path of the two filr, length of the 2 files, and if they are equal
	print(file1, "\n", 
       	  file2, "\n",
		    "equal: ", df1.equals(df2), "\n\n")


#########################à BIO TAGGER ########################################

oTag = "O"  
types = set()

files = {
	"wikinews_train.tsv"	: BIO_PATH + "automatic/WN_train.tsv",
	"wikinews_test.tsv"		: BIO_PATH + "automatic/WN_dev.tsv",
	"fiction_train.tsv"		: BIO_PATH + "automatic/FIC_train.tsv",
	"fiction_test.tsv"		: BIO_PATH + "automatic/FIC_dev.tsv",
	"degasperi_train.tsv"	: BIO_PATH + "automatic/ADG_train.tsv",
	"degasperi_test.tsv"	: BIO_PATH + "automatic/ADG_dev.tsv",
	"moro_train.tsv"		: BIO_PATH + "automatic/MR_train.tsv",
	"moro_test.tsv"			: BIO_PATH + "automatic/MR_dev.tsv",
}

count = {}

for file in files:
	with open(os.path.join(nonBIO_PATH, file), "r") as f:
		outFile = files[file]
		count[outFile] = {"sentences": 0, "tags": {}, "tokens": 0}

		sentences = []
		thisSentence = []

		for line in f:
			line = line.strip()
			if len(line) == 0:
				if len(thisSentence) > 0:
					sentences.append(thisSentence)
					thisSentence = []
				continue
			parts = line.split("\t")
			thisSentence.append(parts)
			count[outFile]["tokens"] += 1

		if len(thisSentence) > 0:
			sentences.append(thisSentence)

		count[outFile]["sentences"] = len(sentences)

		for sentence in sentences:
			previousNer = oTag
			for token in sentence:
				ner = token[1]
				newNer = ner
				if ner != oTag:
					if previousNer != ner:
						if ner not in count[outFile]["tags"]:
							count[outFile]["tags"][ner] = 0
						newNer = "B-" + ner
						count[outFile]["tags"][ner] += 1
						types.add(ner)
					else:
						newNer = "I-" + ner
				token[1] = newNer
				previousNer = ner

		with open(outFile, "w") as fw:
			for sentence in sentences:
				for token in sentence:
					fw.write(token[0])
					fw.write("\t")
					fw.write(token[1])
					fw.write("\n")
				fw.write("\n")

#########################à CHECK ########################################
comp = [
	[BIO_PATH + "automatic/WN_train.tsv", 	'./' + BIO_PATH + '/wikinews_train_BIO.tsv'],
	[BIO_PATH + "automatic/WN_dev.tsv", 	'./' + BIO_PATH + '/wikinews_test_BIO.tsv'],
	[BIO_PATH + "automatic/FIC_train.tsv", 	'./' + BIO_PATH + '/fiction_train_BIO.tsv'],
	[BIO_PATH + "automatic/FIC_dev.tsv", 	'./' + BIO_PATH + '/fiction_test_BIO.tsv'],
	[BIO_PATH + "automatic/ADG_train.tsv", 	'./' + BIO_PATH + '/degasperi_train_BIO.tsv'],
	[BIO_PATH + "automatic/ADG_dev.tsv", 	'./' + BIO_PATH + '/degasperi_test_BIO.tsv'],
	[BIO_PATH + "automatic/MR_train.tsv", 	'./' + BIO_PATH + '/moro_train_BIO.tsv'],
	[BIO_PATH + "automatic/MR_dev.tsv", 	'./' + BIO_PATH + '/moro_test_BIO.tsv'],
]

for i in comp:
	check_equal(i[0], i[1])


# Extra

In [ ]:
# Elia valuta se può essere utile, altrimenti eliminiamo
def compute_statistics(dataset_name, dataset, statistics_df):
  '''
  Computes the statistics of the dataset and adds it into an aggregated structure

  Parameters
  ----------
  dataset_name: str
  dataset: DataFrame
  statistics_df: DataFrame

  Returns
  -------
  statistics_df: DataFrame
  '''
  if statistics_df is None:
    columns = ['Dataset', 'Number of tokens', 'Number of unique tokens', 'Entity tag types']
    statistics_df = pd.DataFrame([[dataset_name, len(dataset), len(dataset['Token'].unique()), sorted(dataset['Tag'].unique())]], columns=columns)
  else:
    columns = ['Dataset', 'Number of tokens', 'Number of unique tokens', 'Entity tag types']
    statistics_df = pd.concat([statistics_df, pd.DataFrame([[dataset_name, len(dataset), len(dataset['Token'].unique()), sorted(dataset['Tag'].unique())]], columns=columns)])
  return statistics_df



  # Rimuovi l'hardcoding dei nomi delle colonne

In [ ]:
# Elia valuta se può essere utile, altrimenti eliminiamo
datasets_df = []
dataset_stats_df = None

for dataset_name in dataset_names:
  dataset_df = pd.read_csv(dataset_name, sep='\t', names=['Token', 'Tag'])
  datasets_df.append(dataset_df)
  dataset_stats_df = compute_statistics(dataset_name, dataset_df, dataset_stats_df)

dataset_stats_df.style.hide(axis='index')

In [ ]:
!pwd